In [8]:
#Imports librairies de base
import numpy as np
import pandas as pd

#Bibliothèques pour Cross validation

#Affichage de Graphes
from IPython.display import Markdown, display, HTML
import matplotlib.pyplot as plt

#Information Mutuelle
from sklearn.feature_selection import mutual_info_regression

In [9]:
# 1.1 Charger les données d'apprentissage (Data/passagers.csv) dans un DataFrame nommé train. Afficher 10 exemples.

train = pd.read_csv("./dev/machineLearning/machineLearning/data/Module1/passagers.csv")

train.sample(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
748,749,0,1,"Marvin, Mr. Daniel Warner",male,19.0,1,0,113773,53.1000,D30,S
619,620,0,2,"Gavey, Mr. Lawrence",male,26.0,0,0,31028,10.5000,NaN,S
653,654,1,3,"O'Leary, Miss. Hanora ""Norah""",female,NaN,0,0,330919,7.8292,NaN,Q
103,104,0,3,"Johansson, Mr. Gustaf Joel",male,33.0,0,0,7540,8.6542,NaN,S
708,709,1,1,"Cleaver, Miss. Alice",female,22.0,0,0,113781,151.5500,NaN,S
649,650,1,3,"Stanley, Miss. Amy Zillah Elsie",female,23.0,0,0,CA. 2314,7.5500,NaN,S
124,125,0,1,"White, Mr. Percival Wayland",male,54.0,0,1,35281,77.2875,D26,S
212,213,0,3,"Perkin, Mr. John Henry",male,22.0,0,0,A/5 21174,7.2500,NaN,S
464,465,0,3,"Maisner, Mr. Simon",male,NaN,0,0,A/S 2816,8.0500,NaN,S
422,423,0,3,"Zimmerman, Mr. Leo",male,29.0,0,0,315082,7.8750,NaN,S


In [10]:
# 1.2 Charger les données d'apprentissage (Data/test.csv) dans un Dataframe nommé test. Afficher 10 exemples.

test = pd.read_csv('./dev/machineLearning/machineLearning/data/Module1/test.csv')

test.sample(10)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
50,942,1,"Smith, Mr. Lucien Philip",male,24.00,1,0,13695,60.0000,C31,S
64,956,1,"Ryerson, Master. John Borie",male,13.00,2,2,PC 17608,262.3750,B57 B59 B63 B66,C
214,1106,3,"Andersson, Miss. Ida Augusta Margareta",female,38.00,4,2,347091,7.7750,NaN,S
73,965,1,"Ovies y Rodriguez, Mr. Servando",male,28.50,0,0,PC 17562,27.7208,D43,C
310,1202,3,"Cacic, Mr. Jego Grga",male,18.00,0,0,315091,8.6625,NaN,S
296,1188,2,"Laroche, Miss. Louise",female,1.00,1,2,SC/Paris 2123,41.5792,NaN,C
354,1246,3,"Dean, Miss. Elizabeth Gladys Millvina""""",female,0.17,1,2,C.A. 2315,20.5750,NaN,S
290,1182,1,"Rheims, Mr. George Alexander Lucien",male,NaN,0,0,PC 17607,39.6000,NaN,S
101,993,2,"Weisz, Mr. Leopold",male,27.00,1,0,228414,26.0000,NaN,S
383,1275,3,"McNamee, Mrs. Neal (Eileen O'Leary)",female,19.00,1,0,376566,16.1000,NaN,S


In [11]:
# 1.3 Afficher les informations des deux DataFrames train et Test.

train.info()

test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass  

In [12]:
# 1.4 Calculer le taux global de survie

survived = train['Survived'].mean() * 100

print(round(survived, 2), "de pourcentage de survie.")


38.38 de pourcentage de survie.


In [13]:
# Question 1 : Quel travail voyez-vous pour que les données soient prêtes à faire tourner dans un modèle ?
# Retravailler les données afin de ne plus afficher des "Nan" dans les colonnes.

# Question 2 : Quelle différence y a-t-il entre les données train et les données test ?
# Il n'y a pas de colonne de survie dans les données test.

# Question 3 : Quel est le rôle de ces deux sets de données ?


In [14]:
# 2.1 Analyser l'effet de la variable Classe sur le taux de survie, en affichant le % de survie par classe

survived_byclass = train.groupby(['Pclass'])['Survived'].mean() * 100

print(survived_byclass)

Pclass
1    62.962963
2    47.282609
3    24.236253
Name: Survived, dtype: float64


In [15]:
# 2.2 Analyser l'effet de la variable Genre (Sex) sur le taux de survie, en affichant le % de survie par genre

survived_bysex = train.groupby(['Sex'])['Survived'].mean() * 100

print(survived_bysex)


Sex
female    74.203822
male      18.890815
Name: Survived, dtype: float64


In [16]:
# 2.3 Calculer une nouvelle colonne FamilySize contenant la taille de la famille, pour les DataFrame train et Test.
# Afficher pour 10 exemples l'Id du passager, le nom, l'age, le sexe et la taille de la famille.

def size_family(row):
    return row['SibSp'] + row['Parch'] + 1

train['FamilySize'] = train.apply(size_family, axis=1)
test['FamilySize'] = test.apply(size_family, axis=1)
print(train[['PassengerId', 'Name', 'Age', 'Sex', 'FamilySize']].sample(10))

     PassengerId                                             Name   Age  \
679          680               Cardeza, Mr. Thomas Drake Martinez  36.0   
879          880    Potter, Mrs. Thomas Jr (Lily Alexenia Wilson)  56.0   
417          418                    Silven, Miss. Lyyli Karoliina  18.0   
231          232                         Larsson, Mr. Bengt Edvin  29.0   
827          828                            Mallet, Master. Andre   1.0   
460          461                              Anderson, Mr. Harry  48.0   
192          193  Andersen-Jensen, Miss. Carla Christine Nielsine  19.0   
492          493                       Molson, Mr. Harry Markland  55.0   
165          166  Goldsmith, Master. Frank John William "Frankie"   9.0   
470          471                                Keefe, Mr. Arthur   NaN   

        Sex  FamilySize  
679    male           2  
879  female           2  
417  female           3  
231    male           1  
827    male           3  
460    male       

In [17]:
# 2.4 Visualisez le nombre de personnes et le taux de survie par taille de famille.

survived_byfamilysize= round(train.groupby(['FamilySize'])['Survived'].mean() * 100, 2)

print(survived_byfamilysize)

FamilySize
1     30.35
2     55.28
3     57.84
4     72.41
5     20.00
6     13.64
7     33.33
8      0.00
11     0.00
Name: Survived, dtype: float64


In [18]:
# 2.5 Calculer une nouvelle colonne IsAlone indiquant si le passager est seul ou pas, pour les DataFrame train et Test. Afficher pour 10 exemples l'Id du passager, le nom, l'age, le sexe et le status IsAlone.

def is_alone(row):
    if row['FamilySize'] == 1 :
        return 1
    else :
        return 0

train['isAlone'] = train.apply(is_alone, axis=1)
test['isAlone'] = test.apply(is_alone, axis=1)

print(train[['PassengerId', 'Name', 'Age', 'Sex', 'isAlone', 'FamilySize']].sample(10))

     PassengerId                                               Name   Age  \
575          576                               Patchett, Mr. George  19.0   
696          697                                   Kelly, Mr. James  44.0   
658          659                       Eitemiller, Mr. George Floyd  23.0   
814          815                         Tomlin, Mr. Ernest Portage  30.5   
68            69                    Andersson, Miss. Erna Alexandra  17.0   
274          275                         Healy, Miss. Hanora "Nora"   NaN   
330          331                                 McCoy, Miss. Agnes   NaN   
427          428  Phillips, Miss. Kate Florence ("Mrs Kate Louis...  19.0   
233          234                     Asplund, Miss. Lillian Gertrud   5.0   
205          206                         Strom, Miss. Telma Matilda   2.0   

        Sex  isAlone  FamilySize  
575    male        1           1  
696    male        1           1  
658    male        1           1  
814    male 

In [19]:
# 2.6 Visualisez le nombre de personnes et le taux de survie par statut seul ou pas.

survived_aloneOrNot = round(train.groupby(['isAlone'])['Survived'].mean() * 100, 2)

print(survived_aloneOrNot)

isAlone
0    50.56
1    30.35
Name: Survived, dtype: float64


In [20]:
# 2.7 Calculer l'information mutuelle sur les données train.
# Attention : Il y a des données continues et des données discrètes
# Ignorer les colonnes Name, Ticket et Cabin



In [21]:
# 2.8 Afficher dans un graphique les scores d'information mutuelle par feature


In [22]:
# Question 1 : La variable Classe est-elle utile pour prévoir la survie ? Pourquoi ?

# Question 2 : Même question pour la variable Sex

# Question 3 : Comment traiter les variables liées à la famille du passager ?

In [23]:
# 3.1 La variable "Embarked" possède des valeurs nulles, compléter par la valeur majoritaire (S).

train['Embarked'].fillna('S', inplace=True)
test['Embarked'].fillna('S', inplace=True)

print(train[['PassengerId', 'Name', 'Age', 'Sex', 'Embarked']].sample(10))

     PassengerId                           Name   Age     Sex Embarked
211          212     Cameron, Miss. Clear Annie  35.0  female        S
522          523             Lahoud, Mr. Sarkis   NaN    male        C
217          218   Jacobsohn, Mr. Sidney Samuel  42.0    male        S
773          774                Elias, Mr. Dibo   NaN    male        C
116          117           Connors, Mr. Patrick  70.5    male        Q
729          730  Ilmakangas, Miss. Pieta Sofia  25.0  female        S
795          796             Otter, Mr. Richard  39.0    male        S
76            77              Staneff, Mr. Ivan   NaN    male        S
218          219          Bazzani, Miss. Albina  32.0  female        C
175          176         Klasen, Mr. Klas Albin  18.0    male        S


In [24]:
# 3.2 Afficher l'influence du port d'embarquement sur la Survie

survived_embarked = round(train.groupby(['Embarked'])['Survived'].mean() * 100, 2)

print(survived_embarked)

Embarked
C    55.36
Q    38.96
S    33.90
Name: Survived, dtype: float64


In [25]:
# 3.3 La variable "Fare" (prix du ticket) possède des valeurs manquantes. Remplacer ces valeurs par la valeur médiane.

train['Fare'].fillna(train['Fare'].median(), inplace=True)
test['Fare'].fillna(test['Fare'].median(), inplace=True)

print(train[['PassengerId', 'Name', 'Age', 'Sex', 'Fare']].sample(10))

     PassengerId                                             Name   Age  \
492          493                       Molson, Mr. Harry Markland  55.0   
869          870                  Johnson, Master. Harold Theodor   4.0   
52            53         Harper, Mrs. Henry Sleeper (Myna Haxtun)  49.0   
186          187  O'Brien, Mrs. Thomas (Johanna "Hannah" Godfrey)   NaN   
458          459                              Toomey, Miss. Ellen  50.0   
633          634                    Parr, Mr. William Henry Marsh   NaN   
74            75                                    Bing, Mr. Lee  32.0   
564          565                   Meanwell, Miss. (Marion Ogden)   NaN   
764          765                           Eklund, Mr. Hans Linus  16.0   
293          294                              Haas, Miss. Aloisia  24.0   

        Sex     Fare  
492    male  30.5000  
869    male  11.1333  
52   female  76.7292  
186  female  15.5000  
458  female  10.5000  
633    male   0.0000  
74     male  

In [26]:
# 3.4 Créer une variable CategoricalFare Découpant la valeur de ticket en 4 catégories de tailles à peu près égales.

categoricalFare = pd.cut()

SyntaxError: invalid syntax (195668231.py, line 3)

In [ ]:
# 3.5 Analyser l'effet de ces catégories de prix de ticket sur la survie.

In [ ]:
# 3.6 Combien de valeurs d'Age manquent ? Complète par des valeurs entières aléatoires entre (moyenne - écart type) et (moyenne + écart type).


In [ ]:
# 3.7 Créer une variable CategoricalAge Découpant la valeur d'age en 5 catégories égales.


In [ ]:
# 3.8 Analyser l'effet de ces catégories d'age sur la survie.

In [ ]:
# Question 1 : Classer les variables par ordre d'intérêt

In [ ]:
# 4.1 : Le nom permet de retrouver le titre du passager. Trouver les titres lorsqu'ils existent.

In [ ]:
# 4.2 : En utilisant la fonction pandas crosstab, afficher pour train le nombre d'occurrences par titres (en ligne) et par sexe (en colonne)


In [ ]:
#4.3 : Remplacer 'Melle et 'Ms' par 'Miss'
#      Remplacer 'Mme' par 'Mrs'
# Créer une valeur "Rare" pour les autres titres avec moins de 10 occurrences.
# Calculer ensuite le taux de survie par titre


In [ ]:
# 5.2 : Attribuer une valeur pour les possibilités de la variable "Title" ('Master','Miss','Mr','Mrs','Rare'). (Question optionnel 4 nécessaires)

In [ ]:
# 5.3 Remplacer les titres manquant par 0, et attribuer à la colonne le type entier. (Question optionnelle 4 nécessaires)


In [ ]:
# 5.4 Attribuer une valeur numérique aux 4 catégories de la variable Fare (question 3.6)

In [ ]:
# 5.5 Attribuer une valeur numérique au Port d'embarquement (Embarked)

In [ ]:
# 5.6 Supprimer les features inutiles


In [ ]:
# 5.7 Afficher les infos pour les deux datasets


In [ ]:
# 5.8 Sauver les deux datasets dans des fichiers csv. Éviter d'écrire un index (n° de ligne)


In [ ]:
# Question : Sur combien de questions avez-vous utilisé les conseils ?
